In [1]:
from PIL import Image
import numpy as np
from collections import Counter
import heapq

# Load the image
img_path = 'pisang2/pisang2_grayscale.jpg'
img = Image.open(img_path)

# Convert image to grayscale
img_gray = img.convert("L")

# Get the pixel values
pixels = np.array(img_gray).flatten()

In [2]:
# Calculate frequency of each pixel value
freq = Counter(pixels)

# Huffman tree node class
class Node:
    def __init__(self, left=None, right=None, root=None):
        self.left = left
        self.right = right
        self.root = root

    def children(self):
        return (self.left, self.right)

    def __lt__(self, other):
        return self.root < other.root

In [3]:
# Build Huffman Tree
def build_huffman_tree(frequencies):
    heap = [Node(root=freq, left=symbol) for symbol, freq in frequencies.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        node = Node(left=left, right=right, root=left.root + right.root)
        heapq.heappush(heap, node)
    return heap[0]

In [4]:
# Assign codes to leaves of Huffman tree
def assign_codes(node, prefix="", code={}):
    if isinstance(node.left, Node):
        assign_codes(node.left, prefix + "0", code)
    else:
        code[node.left] = prefix or "0"
    if isinstance(node.right, Node):
        assign_codes(node.right, prefix + "1", code)
    else:
        code[node.right] = prefix or "0"
    return code

In [5]:
# Build the tree and assign codes
huffman_tree = build_huffman_tree(freq)
huffman_codes = assign_codes(huffman_tree)

# Display some of the codes
list(huffman_codes.items())[:10]

[(103, '000000'),
 (None, '111111'),
 (108, '000001'),
 (75, '0000100'),
 (147, '000010100'),
 (153, '000010101'),
 (198, '000010110'),
 (214, '000010111'),
 (106, '000011'),
 (107, '000100')]

In [6]:
# Encode the image using Huffman codes
encoded_image = ''.join(huffman_codes[pixel] for pixel in pixels)

# Calculate the total number of bits in the original image (8 bits per pixel)
original_bits = len(pixels) * 8

# Calculate the total number of bits in the encoded image
encoded_bits = len(encoded_image)

# Calculate compression ratio
compression_ratio = original_bits / encoded_bits

original_bits, encoded_bits, compression_ratio

(1384448, 1160003, 1.1934865685692193)

In [7]:
# Decode Huffman Codes back to pixel values
def decode_huffman(encoded_str, huffman_codes):
    reverse_huffman_codes = {v: k for k, v in huffman_codes.items()}
    current_code = ''
    decoded_pixels = []
    for bit in encoded_str:
        current_code += bit
        if current_code in reverse_huffman_codes:
            decoded_pixels.append(reverse_huffman_codes[current_code])
            current_code = ''
    return decoded_pixels

# Decode the encoded image
decoded_pixels = decode_huffman(encoded_image, huffman_codes)

# Create an image from the decoded pixel values
decoded_image = Image.new('L', img_gray.size)
decoded_image.putdata(decoded_pixels)
decoded_image_path = 'pisang2/pisang2_compress.jpg'
decoded_image.save(decoded_image_path)

decoded_image_path


'pisang2/pisang2_compress.jpg'